<a href="https://colab.research.google.com/github/gihantha-sanjana/CMP7005-Assingment-Work-Repo/blob/main/Beijing%20Multi-Site%20Air%20Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries needed.**

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# **Mount the Google Drive.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# **Direct to the DataSet Folder Path.**

In [ ]:
import os
# Change to the desired directory
os.chdir('/content/drive/My Drive/ColabNotebooks/DatasetCMP7005')

# Verify the current directory
print("Current Directory:", os.getcwd())

# **Display All CSV files in my Google Drive Folder**

In [ ]:
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

In [ ]:
df_list = [pd.read_csv(file) for file in csv_files]

In [ ]:
# Merge the 7 Data Files into One Main CSV file
df_merged = pd.concat(df_list, axis=0, ignore_index=True)

# **Merge the DataSet files into One.**

In [ ]:
df_merged

# **Check for null values, data types, and overall structure.**

In [ ]:
df_merged.info()

# **Rename the Dataset Column Names**

In [ ]:
df_merged.rename(columns={'No': 'NO', 'year': 'YEAR', 'month': 'MONTH', 'day':'DAY','hour':'HOUR','DEWP':'DEW_POINT','wd':'WD','WSPM':'WS','station':'STATION'}, inplace=True)

In [ ]:
# Check after Rename columns
df_merged.head()

# **Converting Object dtypes to category dtype for better Memory-effeciency and Performance.**

In [ ]:
df_merged['WD'] = df_merged['WD'].astype('category')
df_merged['STATION'] = df_merged['STATION'].astype('category')

In [ ]:
# Check after Converting
df_merged.dtypes

In [ ]:
# Check Null values as Precentage
df_merged.isna().sum()/len(df_merged)*100

# **Combined the Year, Month, Day and Hour Columns as a One Column in Dataset becuase we can do EDA easily by doing this.**

In [ ]:
#Combine the Columns
df_merged['DATETIME'] = pd.to_datetime(df_merged[['YEAR', 'MONTH', 'DAY', 'HOUR']])

#Remove the original columns
df_merged = df_merged.drop(columns=['YEAR', 'MONTH', 'DAY', 'HOUR'])

df_merged.head()

In [ ]:
# Check the Datetime column add and Remove current columns call YEAR,MONTH,DAY and HOUR
df_merged.info()

# **Check the DataSet Rows**

In [ ]:
df_merged.shape[0]

# **Check the DataSet Columns**

In [ ]:
df_merged.shape[1]

In [ ]:
df_merged.isna().sum()

# **Checking the Precentage of Missing Values in DataSet**

In [ ]:
df_merged.isna().sum()/len(df_merged)*100

# **check the Missing Values DataType**

In [ ]:
df_merged.isna().sum().dtypes

# **Checking the Missing Values in my Dataset using HeatMap in here Every yellow line indicates true it meaning where we have null values.**

In [ ]:
import seaborn as sns
sns.heatmap(df_merged.isnull(), yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
# This also view the Missing values in Our Dataset
import missingno as msno
msno.matrix(df_merged)


# **Drop the No Column becuase it is not Important.**

In [ ]:
df_merged.drop('NO', axis=1, inplace=True)

In [ ]:
# Check the NO column is droped from the Head
df_merged.head()

# **Total missing values per variable**

In [ ]:
data = df_merged.copy()
data['DATETIME'] = pd.to_datetime(data['DATETIME'])
data.set_index('DATETIME', inplace=True)

# **Check Missing Year wise Carbon Monoxide Value using plot**

In [ ]:
mis_co = data['CO'].isnull().astype(int)
plt.figure(figsize=(6, 3))
mis_co.resample('D').sum().plot(title="Missing Carbon Monoxide Over Time", ylabel="Count of Missing Values", xlabel="Date")
plt.show()

# **Check Total Missing Values Per Variable using Bar Plot.**

In [ ]:
data.isnull().sum().plot(kind='bar', title='Total Missing Values by Variable', xlabel='Variables', ylabel='Count')
plt.show()

In [ ]:
mis_pm25 = data['PM2.5'].isnull().astype(int)
plt.figure(figsize=(8, 4))
mis_pm25.resample('D').sum().plot(title="Missing PM2.5 Over Time", ylabel="Count of Missing Values", xlabel="Date")
plt.show()


In [ ]:
mis_pm10 = data['PM10'].isnull().astype(int)
plt.figure(figsize=(8, 4))
mis_pm10.resample('D').sum().plot(title="Missing PM10 Over Time", ylabel="Count of Missing Values", xlabel="Date")
plt.show()


# **Filling null values using**

# **I use Interpolation becuase my data set is Time-Series data.**

In [ ]:
pollutants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
data[pollutants] = data[pollutants].interpolate(method='time')
print(data[pollutants].isnull().sum())

In [ ]:
columns_to_interpolate = ['TEMP', 'PRES', 'DEW_POINT']
data[columns_to_interpolate] = data[columns_to_interpolate].interpolate(method='linear')
print(data[columns_to_interpolate].isnull().sum())

# **For fill the Rain and Wind Speed I used Fo**

In [ ]:
data['RAIN'] = data['RAIN'].ffill()
data['RAIN'] = data['RAIN'].bfill()
data['WS'] = data['WS'].ffill()
data['WS'] = data['WS'].bfill()

# **Verify the null values are fill.**

In [ ]:
print(data.isnull().sum())

# **Check the Null Values Summary in Data Set**

In [ ]:
data.isna().sum()

# **Convert Wind Direction into Quater-Winds**

In [ ]:
direction_mapping = {
    'N': 0,         # North
    'NNE': 22.5,    # North-North-East
    'NE': 45,       # North-East
    'ENE': 67.5,    # East-North-East
    'E': 90,        # East
    'SE': 135,      # South-East
    'S': 180,       # South
    'SSW': 202.5,   # South-South-West
    'SW': 225,      # South-West
    'WSW': 247.5,   # West-South-West
    'W': 270,       # West
    'WNW': 292.5,   # West-North-West
    'NW': 315,      # North-West
    'NNW': 337.5    # North-North-West
}

# Apply the mapping to the 'Wind Direction' column
data['WD'] = data['WD'].map(direction_mapping)

# **Wind Direction Column fill with Previous Valid Value**

In [ ]:
data['WD'] = data['WD'].ffill()

In [ ]:
data.isna().sum()

# **Now check the Dataset still have Null values.**

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(data.isnull(), cbar=False, cmap="viridis")
plt.title("Heatmap of Missing Values")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.dtypes

# **Feature Engineering**

In [ ]:
import numpy as np

def calculate_humidity(temp, dew_point):
    # Calculate relative humidity using the dew point and temperature
    numerator = np.exp((17.27 * temp) / (temp + 237.7))
    denominator = np.exp((17.27 * dew_point) / (dew_point + 237.7))
    return 100 * (denominator / numerator)

# Apply to your data
data['HUMIDITY'] = data.apply(lambda row: calculate_humidity(row['TEMP'], row['DEW_POINT']), axis=1)

# View the calculated humidity
print(data[['TEMP', 'DEW_POINT', 'HUMIDITY']].head())


# **Calculate AQI and AQI Category.**

In [ ]:
# Define AQI breakpoints
breakpoints = {
    'PM2.5': {
        'low': [0, 12, 35.4, 55.4, 150, 250],
        'high': [12, 35.4, 55.4, 150, 250, 500],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    },
    'PM10': {
        'low': [0, 54, 154, 254, 354, 424],
        'high': [54, 154, 254, 354, 424, 604],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    },
    'NO2': {
        'low': [0, 53, 100, 360, 649, 1249],
        'high': [53, 100, 360, 649, 1249, 2049],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    },
    'CO': {
        'low': [0, 4.4, 9.4, 12.4, 15.4, 30.4],
        'high': [4.4, 9.4, 12.4, 15.4, 30.4, 50.4],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    },
    'SO2': {
        'low': [0, 35, 75, 185, 304, 604],
        'high': [35, 75, 185, 304, 604, 1004],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    },
    'O3': {
        'low': [0, 54, 70, 85, 105, 200],
        'high': [54, 70, 85, 105, 200, 300],
        'low_aqi': [0, 50, 100, 150, 200, 300],
        'high_aqi': [50, 100, 150, 200, 300, 500]
    }
}

# AQI Categories
aqi_categories = {
    (0, 50): "Good",
    (51, 100): "Moderate",
    (101, 150): "Unhealthy for Sensitive Groups",
    (151, 200): "Unhealthy",
    (201, 300): "Very Unhealthy",
    (301, 500): "Hazardous"
}

# Function to calculate AQI and category
def calculate_aqi(concentration, pollutant):
    if pd.isnull(concentration):
        return None, None  # Return None if the concentration is NaN

    low_bp = breakpoints[pollutant]['low']
    high_bp = breakpoints[pollutant]['high']
    low_aqi = breakpoints[pollutant]['low_aqi']
    high_aqi = breakpoints[pollutant]['high_aqi']

    # Find the breakpoint range
    for i in range(len(low_bp)):
        if low_bp[i] <= concentration <= high_bp[i]:
            I_low = low_aqi[i]
            I_high = high_aqi[i]
            BP_low = low_bp[i]
            BP_high = high_bp[i]
            # Calculate AQI using interpolation
            aqi = ((I_high - I_low) / (BP_high - BP_low)) * (concentration - BP_low) + I_low
            # Determine the AQI category
            for (low, high), category in aqi_categories.items():
                if low <= aqi <= high:
                    return round(aqi, 2), category
    return None, None  # If no range matches

# Function to calculate max AQI and category across pollutants
def calculate_max_aqi_and_category(row):
    pollutants = ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3']
    aqi_values = []
    categories = []

    # Calculate AQI and category for each pollutant
    for pollutant in pollutants:
        aqi, category = calculate_aqi(row[pollutant], pollutant)
        if aqi is not None:
            aqi_values.append(aqi)
            categories.append(category)

    # Determine maximum AQI and its category
    if aqi_values:
        max_aqi = max(aqi_values)
        max_category = categories[aqi_values.index(max_aqi)]
        return max_aqi, max_category
    else:
        return None, None

# Apply AQI calculations to the dataset
data['MAX_AQI'], data['AQI_CATEGORY'] = zip(*data.apply(calculate_max_aqi_and_category, axis=1))

# Display the results
print(data[['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'MAX_AQI', 'AQI_CATEGORY']].head())

In [ ]:
data.head()

In [ ]:
data.describe()

Save the Clean Dataset to the Google Drive.

# **Handle Outliers in My Dataset.**

# **plot boxplot to check outliers in my dataset selected columns**

In [ ]:
# # Function to detect and summarize outliers
# def detect_outliers_iqr(df, column):
#     q1 = df[column].quantile(0.25)
#     q3 = df[column].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr

#     outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
#     print(f"{column}: Found {len(outliers)} outliers")
#     return lower_bound, upper_bound

# # Detect outliers in RAIN and WS
# rain_lower, rain_upper = detect_outliers_iqr(data, 'RAIN')


In [ ]:
# data['RAIN'] = data['RAIN'].clip(lower=rain_lower, upper=rain_upper)

In [ ]:
# # Summary statistics after handling
# print(data[['RAIN']].describe())

# # Visualization
# plt.figure(figsize=(12, 5))

# # RAIN distribution
# plt.subplot(1, 2, 1)
# sns.boxplot(data=data, x='RAIN')
# plt.title('RAIN After Handling Outliers')

# # # WS distribution
# # plt.subplot(1, 2, 2)
# # sns.boxplot(data=data, x='WS')
# # plt.title('WS After Handling Outliers')

# plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot boxplots for numerical columns
numerical_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3','MAX_AQI','WS']

plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_columns):
    plt.subplot(2, 4, i+1)
    sns.boxplot(y=data[col], color='skyblue')
    plt.title(f"Boxplot of {col}")
plt.tight_layout()
plt.show()


# **check the each column number of outliers using IRQ function.**

In [ ]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    print(f"{column}: Found {len(outliers)} outliers")
    return outliers

# Check outliers for each column
for col in numerical_columns:
    detect_outliers_iqr(data, col)


# **use cap method to replace my outliers with the nearest valid values**

In [ ]:
# Capping outliers using IQR method
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# Cap outliers for all numerical columns
for col in numerical_columns:
    data = cap_outliers(data, col)


# **Check again using boxplot still we have the outliers in my dataset.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot boxplots for numerical columns
numerical_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO','MAX_AQI','WS']

plt.figure(figsize=(16, 10))
for i, col in enumerate(numerical_columns):
    plt.subplot(2, 4, i+1)
    sns.boxplot(y=data[col], color='skyblue')
    plt.title(f"Boxplot of {col}")
plt.tight_layout()
plt.show()


In [ ]:
data.head()

In [ ]:
data.to_csv("/content/drive/MyDrive/ColabNotebooks/DatasetCMP7005/MergeDataSet/Preprocessed-Beijing-Multi-Site-Air-Quality.CSV", index=False)

# **EDA**

In [ ]:
print("Dataset Overview:")
print(data.head())

In [ ]:
print("Statistical Summary:")
print(data.describe().T)


In [ ]:
categorical_cols = data.select_dtypes(include=['category', 'object']).columns
for col in categorical_cols:
    print(f"Value counts for {col}:\n", data[col].value_counts(), "\n")


# **Distribution Analysis for Numeric Features**

In [ ]:
numeric_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEW_POINT', 'RAIN', 'WS', 'HUMIDITY', 'MAX_AQI']
data[numeric_columns].hist(figsize=(15, 12), bins=30, color='skyblue', edgecolor='black')
plt.suptitle('Distribution of Numeric Features', fontsize=16)
plt.show()

# **Categorical Feature Analysis**

In [ ]:
plt.figure(figsize=(10, 6))
data['AQI_CATEGORY'].value_counts().plot(kind='bar', color='purple', edgecolor='black')
plt.title('Count of AQI Categories')
plt.xlabel('AQI Categories')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

# **Correlation Analysis**

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data[numeric_columns].corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

# **Station-Specific Analysis**

In [ ]:
station_group = data.groupby('STATION')[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'MAX_AQI']].mean()
station_group.plot(kind='bar', figsize=(18, 8), stacked=True, colormap='viridis')
plt.title('Average Pollutants by Station')
plt.ylabel('Average Levels')
plt.xlabel('Station')
plt.legend(loc='upper right')
plt.show()

In [ ]:
resampled_data = data[['PM2.5', 'PM10', 'MAX_AQI']].resample('M').mean()
resampled_data.plot(figsize=(15, 6))
plt.title('Monthly Trends of Pollutants')
plt.ylabel('Average Levels')
plt.show()


# **Relationship Between Pollutants**

In [ ]:
sns.pairplot(data[numeric_columns + ['AQI_CATEGORY']], hue='AQI_CATEGORY', palette='husl', diag_kind='kde', height=2)
plt.suptitle('Relationships Between Key Pollutants', y=1.02)
plt.show()

KeyboardInterrupt: 

# **Wind Direction Analysis**

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['WD'], bins=16, kde=True, color='purple')
plt.title('Distribution of Wind Direction')
plt.xlabel('Wind Direction (Degrees)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
for col in categorical_cols:
    plt.figure(figsize=(12, 6))
    sns.countplot(data=data, x=col, hue="MAX_AQI", palette="viridis")
    plt.title(f"Count of {col}")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Example: Relationship between temperature and pollution level
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x="TEMP", y="MAX_AQI", hue="AQI_CATEGORY", palette="viridis")
plt.title("Temperature vs Pollution Level")
plt.show()


In [ ]:
# Example: Pollution level across AQI categories
if 'AQI_CATEGORY' in data.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=data, x="AQI_CATEGORY", y="MAX_AQI", palette="Set3")
    plt.title("Pollution Level by AQI Category")
    plt.xticks(rotation=45)
    plt.show()
